In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [81]:
DATA_PATH = "./data/ratings.csv"

In [6]:
dtype = [("userId", np.int32), ("movieId", np.int32), ("rating", np.float32)]
dataset = pd.read_csv("data/ratings.csv", usecols=range(3), dtype=dict(dtype))

## 计算BaseLine
代价函数:
$$
Cost=\sum_{u,i\in R}(r_{ui}-\mu-b_u-b_i)^2 + \lambda*(\sum_u {b_u}^2 + \sum_i {b_i}^2)
$$  
SGD（随机梯度下降法）:
\begin{split}b_u&:=b_u + \alpha*((r_{ui}-\mu-b_u-b_i) -\lambda*b_u)  \\&:=b_u + \alpha*(error - \lambda*b_u) \\\\b_i&:=b_i + \alpha*((r_{ui}-\mu-b_u-b_i) -\lambda*b_i)\\&:=b_i + \alpha*(error -\lambda*b_i)\end{split}
ASL（交替最小二乘法优化）:  
$$
 b_u := \cfrac {\sum_{u,i\in R}(r_{ui}-\mu-b_i)}{\lambda_1 + |R(u)|}
$$

$$
b_i := \cfrac {\sum_{u,i\in R}(r_{ui}-\mu-b_u)}{\lambda_2 + |R(i)|}
$$

### 数据集的划分

In [88]:
def data_split(data_path, x = 0.8, random = False):
    """划分数据集合"""
    
    print("开始划分数据......")
    dtype = {"userId": np.int32, "movieId": np.int32, "rating": np.float32}
    # 加载数据，我们只用前三列数据，分别是用户ID，电影ID，已经用户对电影的对应评分
    ratings = pd.read_csv(data_path, dtype=dtype, usecols=range(3))
    
    testset_index = []
    
    for uid in ratings.groupby("userId").any().index:  # 获取index索引
        use_rating_data = ratings.where(ratings["userId"]==uid).dropna()
        if random:
            # shuffle
            index = list(use_rating_data.index)
            np.random.shuffle(index)
            _index = round(len(use_rating_data)*x)
            testset_index += list(index[_index:]) # 取出1-x的数据做测试
        else:
            index = round(len(use_rating_data)*x)
            testset_index += list(use_rating_data.index.values[index:])
            
    testset = ratings.loc[testset_index]
    trainset = ratings.drop(testset_index)
    print("完成数据集的切分......")
    
    return trainset, testset

In [89]:
train_set, test_set = data_split(DATA_PATH)

开始划分数据......
完成数据集的切分......


In [87]:
len(train_set)/len(test_set)

4.000793493354493

### 评价指标（连续性数值的评价预测）
MSE、RMSE、MAE、RMAE

In [92]:
def accuracy(predict_results, method="all"):
    
    def rmse(predict_results):
        """均方根误差"""
        
        length = 0
        _rmse_sum = 0
        for uid, iid, real_rating, pre_rating in predict_results:
            length += 1
            _rmse_sum += (pre_rating - real_rating)**2
        
        return round(np.sqrt(_rmse_sum/length), 4)
    
    def mae(predict_results):
        """平均绝对误差"""
        
        length = 0
        _mae_sum = 0
        for uid, iid, real_rating, pre_rating in predict_results:
            length += 1
            _mae_sum += abs(pre_rating - real_rating)
        
        return round(_mae_sum/length, 4)
    
    def rmse_mae(predict_results):
        
        length = 0
        _mae_sum = 0
        _rmse_sum = 0
        for uid, iid, real_rating, pre_rating in predict_results:
            length += 1
            _rmse_sum += (pre_rating - real_rating)**2
            _mae_sum += abs(pre_rating - real_rating)
        
        return round(np.sqrt(_rmse_sum/length), 4), round(_mae_sum/length, 4)
                            
    if method.lower() == 'rmse':
        rmse(predict_results)
    elif method.lower() == "mae":
        mae(predict_results)
    else:
        return rmse_mae(predict_results)

### 封装实现

In [150]:
class BaselineCFBySGD:
    
    def __init__(self, num_epochs, alpha=0.1, reg=0.1, reg_bu=0.1, reg_bi=0.1, columns=["uid", "iid", "rating"]):
        
        self.num_epochs = num_epochs 
        self.alpha = alpha  # learning rate
        self.reg = reg  # L2
        self.columns = columns
        
        self.reg_bi = reg_bi
        self.reg_bu = reg_bu
    
    def fit(self, datasets, metric="SGD"):
        """训练参数"""
        
        self.datasets = datasets
        # 用户评分表
        self.users_ratings = datasets.groupby(self.columns[0]).agg([list])[[self.columns[1], self.columns[2]]]
        # 物品评分表
        self.items_ratings = datasets.groupby(self.columns[1]).agg([list])[[self.columns[0], self.columns[2]]]
                          
        # 全局平均分
        self.global_mean = self.datasets[self.columns[2]].mean()
        if metric.lower() == "sgd":
            self.bu, self.bi = self.sgd()
        elif metric.lower() == "als":
            self.bu, self.bi = self.als()
        else:
            print("metric is error!")
        
    def sgd(self):
        """使用随机梯度下降，更新参数bu, bi"""
        
        # 初始化参数值
        bu = dict(zip(self.users_ratings.index, np.zeros(len(self.users_ratings))))
        bi = dict(zip(self.items_ratings.index, np.zeros(len(self.items_ratings))))
        
        # 迭代更新
        for i in range(self.num_epochs):
            print("Epoch:%d"%i)
            for uid, iid, real_rating in self.datasets.itertuples(index=False):
                error = real_rating - (self.global_mean + bu[uid] + bi[iid])
                bu[uid] += self.alpha*(error - self.alpha*bu[uid])
                bi[iid] += self.alpha*(error - self.alpha*bi[iid])
                
        return bu, bi
    
    def als(self):
        """使用交替最小二乘法优化，更新参数bu, bi"""
        
        # 初始化参数值
        bu = dict(zip(self.users_ratings.index, np.zeros(len(self.users_ratings))))
        bi = dict(zip(self.items_ratings.index, np.zeros(len(self.items_ratings))))
        
        # 迭代更新
        for i in range(self.num_epochs):
            print("Epoch:%d"%i)
            
            # 更新bi
            for iid, uids, ratings in self.items_ratings.itertuples(index=True):
                _sum = 0
                for uid, rating in zip(uids, ratings):
                    _sum += rating - self.global_mean - bu[uid]
                bi[iid] = _sum / (self.reg_bi + len(uids))  # len(uids) 物品i的评分数量
                
            # 更新bu
            for uid, iids, ratings in self.users_ratings.itertuples(index=True):
                _sum = 0
                for iid, rating in  zip(iids, ratings):
                    _sum += rating - self.global_mean - bi[iid]
                bu[uid] = _sum / (self.reg_bu + len(iids))
            
        return bu, bi
    
    def predict(self, uid, iid):
        """预测"""
        
        predict_rating = self.global_mean + self.bu[uid] + self.bi[iid]
        
        return predict_rating
    
    def test(self, testset):
        '''预测测试集数据'''
    
        for uid, iid, real_rating in testset.itertuples(index=False):
            try:
                pred_rating = self.predict(uid, iid)
            except Exception as e:
                pass
            else:
                yield uid, iid, real_rating, pred_rating

In [151]:
obj = BaselineCFBySGD(20, 0.1, 0.1, columns=["userId", "movieId", "rating"])
obj.fit(train_set, metric="Als")

Epoch:0
Epoch:1
Epoch:2
Epoch:3
Epoch:4
Epoch:5
Epoch:6
Epoch:7
Epoch:8
Epoch:9
Epoch:10
Epoch:11
Epoch:12
Epoch:13
Epoch:14
Epoch:15
Epoch:16
Epoch:17
Epoch:18
Epoch:19


In [152]:
obj.predict(1, 4)

3.1282720098581

In [153]:
result = obj.test(test_set)

In [154]:
for uid, iid, real, pred in result:
    print(real, pred)
    break

4.0 4.411508363619913


In [155]:
rmse, mae = accuracy(result)

In [156]:
rmse

0.9329

In [157]:
mae

0.7148